In [269]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from scipy.stats import pearsonr

In [271]:
# 加载数据
print("加载数据...")
data = pd.read_stata("./wave3n.dta")
print(f"数据集大小: {data.shape}")
data.head(10)

加载数据...
数据集大小: (21097, 71)


,ID,gender,marry,rural,srh,adlab_c,hibpe,diabe,cancre,lunge,...,exercise,memeory,executive,total_cognition,chronic,ins,pension,iadl,ea001s9,ea001s10
0,010104101001,女性,已婚,农村,一般,0.0,否,是,否,否,...,NaN,5.5,9.0,14.5,是,是,否,0.0,否,NaN
1,010104101002,男性,已婚,农村,一般,0.0,是,否,否,是,...,NaN,3.5,8.0,11.5,是,是,否,0.0,否,NaN
2,010104102001,女性,已婚,农村,一般,0.0,否,否,是,否,...,是,3.5,4.0,7.5,是,是,是,0.0,否,NaN
3,010104102002,男性,已婚,农村,一般,0.0,否,否,否,否,...,是,3.5,8.0,11.5,是,是,是,0.0,否,NaN
4,010104103001,男性,已婚,农村,一般,0.0,是,否,否,否,...,NaN,5.0,9.0,14.0,是,是,否,0.0,否,NaN
5,010104103002,女性,已婚,农村,较差,0.0,否,否,否,否,...,NaN,5.0,5.0,10.0,是,是,否,0.0,否,NaN
6,010104104001,男性,已婚,农村,一般,0.0,否,否,否,否,...,NaN,1.0,9.0,10.0,是,是,是,1.0,否,NaN
7,010104104002,女性,已婚,农村,较差,0.0,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,是,否,0.0,否,NaN
8,010104105001,女性,已婚,农村,较差,1.0,否,否,否,否,...,是,2.5,3.0,5.5,是,是,是,0.0,否,NaN
9,010104105002,男性,已婚,农村,很差,0.0,是,否,否,否,...,是,3.5,9.0,12.5,是,否,NaN,0.0,否,NaN


处理缺失值

In [285]:
# 处理缺失值
print("\n处理缺失值...")
missing_ratio = data.isnull().mean().sort_values(ascending=False)
print("缺失率排序 (前10个):\n", missing_ratio.head(10))

# 设定阈值（缺失率>30%则删除）
threshold = 0.3
high_missing_cols = missing_ratio[missing_ratio > threshold].index.tolist()
data_clean = data.drop(columns=high_missing_cols)
print("删除的高缺失率特征:", high_missing_cols)

# 处理特殊符号（.m, .d, .r）转换为NaN
def convert_special_to_nan(df):
    """处理所有列中的特殊符号，包括数值列"""
    df_copy = df.copy()
    
    # 定义需要替换的特殊符号
    special_values = ['.m', '.d', '.r', '.', '.w', 'p', '.p','.x','.a']
    
    # 遍历所有列
    for col in df_copy.columns:
        # 先将列转换为字符串类型进行替换
        col_str = df_copy[col].astype(str)
        
        # 替换特殊符号为NaN
        for special_val in special_values:
            col_str = col_str.replace(special_val, np.nan)
        
        # 尝试将列转换回数值类型
        try:
            # 如果原列是数值类型，尝试转换回数值
            if df[col].dtype in ['int64', 'float64', 'int32', 'float32']:
                df_copy[col] = pd.to_numeric(col_str, errors='coerce')
            else:
                # 如果是object类型，保持为字符串但替换特殊值
                df_copy[col] = col_str
        except:
            # 如果转换失败，保持为字符串
            df_copy[col] = col_str
    
    return df_copy

# 应用函数处理特殊符号
data_clean = convert_special_to_nan(data_clean)

# 对分类变量用众数填充
categorical_cols = data_clean.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    mode_val = data_clean[col].mode()[0]
    data_clean[col] = data_clean[col].fillna(mode_val)

# 对连续变量用均值填充
numeric_cols = data_clean.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    mean_val = data_clean[col].mean()
    data_clean[col] = data_clean[col].fillna(mean_val)

# 验证是否还有缺失值
print("剩余缺失值数量:", data_clean.isnull().sum().sum())


处理缺失值...
缺失率排序 (前10个):
 ea001s10           1.000000
income_total       0.630753
mdact_c            0.533820
ltact_c            0.533583
vgact_c            0.533346
exercise           0.533251
hhcperc            0.324264
dyslipe            0.214817
total_cognition    0.204342
diabe              0.200028
dtype: float64
删除的高缺失率特征: ['ea001s10', 'income_total', 'mdact_c', 'ltact_c', 'vgact_c', 'exercise', 'hhcperc']
剩余缺失值数量: 0


In [287]:
data_clean.head()

,ID,gender,marry,rural,srh,adlab_c,hibpe,diabe,cancre,lunge,...,age,edu,memeory,executive,total_cognition,chronic,ins,pension,iadl,ea001s9
0,010104101001,女性,已婚,农村,一般,0.0,否,是,否,否,...,50.0,小学以下,5.5,9.0,14.5,是,是,否,0.0,否
1,010104101002,男性,已婚,农村,一般,0.0,是,否,否,是,...,52.0,小学,3.5,8.0,11.5,是,是,否,0.0,否
2,010104102001,女性,已婚,农村,一般,0.0,否,否,是,否,...,61.0,小学以下,3.5,4.0,7.5,是,是,是,0.0,否
3,010104102002,男性,已婚,农村,一般,0.0,否,否,否,否,...,64.0,小学以下,3.5,8.0,11.5,是,是,是,0.0,否
4,010104103001,男性,已婚,农村,一般,0.0,是,否,否,否,...,52.0,小学,5.0,9.0,14.0,是,是,否,0.0,否


检查极端值

In [291]:
# 检查极端值
print("\n检查极端值...")
# 检查年龄的极端值
if 'age' in data_clean.columns:
    print("年龄范围:", data_clean['age'].min(), "至", data_clean['age'].max())
    # 筛选出合理的年龄范围（>65）
    data_clean = data_clean[(data_clean['age'] >= 65)]
    print(f"筛选后数据集大小: {data_clean.shape}")

# 检查收入的极端值
if 'income_total' in data_clean.columns:
    print("收入范围:", data_clean['income_total'].min(), "至", data_clean['income_total'].max())
    # 移除负收入
    data_clean = data_clean[data_clean['income_total'] >= 0]
    print(f"筛选后数据集大小: {data_clean.shape}")


检查极端值...
年龄范围: 65.0 至 115.0
筛选后数据集大小: (6506, 64)


In [293]:
data_clean.to_csv("./wave3n.csv", index=False, encoding='utf-8')
data_clean = pd.read_csv("./wave3n.csv")
data_clean.head()

,ID,gender,marry,rural,srh,adlab_c,hibpe,diabe,cancre,lunge,...,age,edu,memeory,executive,total_cognition,chronic,ins,pension,iadl,ea001s9
0,10104105001,女性,已婚,农村,较差,1.0,否,否,否,否,...,70.0,小学以下,2.500000,3.000000,5.500000,是,是,是,0.0,否
1,10104105002,男性,已婚,农村,很差,0.0,是,否,否,否,...,72.0,中学,3.500000,9.000000,12.500000,是,否,NaN,0.0,否
2,10104107001,男性,已婚,农村,NaN,0.0,是,否,否,否,...,83.0,小学以下,3.400021,8.393857,12.040004,是,是,是,0.0,否
3,10104107002,女性,已婚,农村,NaN,0.0,NaN,NaN,NaN,NaN,...,85.0,小学以下,3.400021,8.393857,12.040004,NaN,是,是,1.0,否
4,10104110002,男性,已婚,农村,较好,0.0,是,否,否,否,...,69.0,小学以下,0.500000,6.000000,6.500000,是,是,是,2.0,否


特征编码

In [295]:
# 定义需要进行特定映射的特征
specific_mapping_features = {
    'gender': {'男性': 1, '女性': 0},
    'marry': {'已婚': 1, '其他': 0},
    'rural': {'农村': 0, '城市': 1}
}

# 对特定特征进行映射
for feature, mapping in specific_mapping_features.items():
    if feature in data_clean.columns:
        data_clean[feature] = data_clean[feature].fillna('').map(mapping)
        print(f"特征 {feature} 的映射结果:\n", data_clean[feature].value_counts())

# 对 edu 进行标签编码
if 'edu' in data_clean.columns:
    label_encoder = LabelEncoder()
    data_clean['edu'] = label_encoder.fit_transform(data_clean['edu'])
    print("edu 的标签编码结果:\n", data_clean['edu'].value_counts())
# 对srh进行编码（很差=0, 较差=1, 一般=2, 较好=3, 很好=4）
if 'srh' in data_clean.columns:
    srh_mapping = {'很差': 0, '较差': 1, '一般': 2, '较好': 3, '很好': 4}
    data_clean['srh'] = data_clean['srh'].map(srh_mapping)
    
    # 检查映射后的值
    print("SRH编码后的值分布:\n", data_clean['srh'].value_counts())

# 对其他非数值型的分类变量进行二值编码（是为1，否为0）
other_categorical_features = [col for col in data_clean.select_dtypes(include=['object']).columns if col not in specific_mapping_features]
for feature in other_categorical_features:
    if feature in data_clean.columns:
        data_clean[feature] = data_clean[feature].fillna('否').map({'是': 1, '否': 0})
        print(f"特征 {feature} 的二值编码结果:\n", data_clean[feature].value_counts())
# 验证是否还有缺失值
print("剩余缺失值数量:", data_clean.isnull().sum().sum())

特征 gender 的映射结果:
 gender
1    3262
0    3244
Name: count, dtype: int64
特征 marry 的映射结果:
 marry
1.0    4722
0.0    1777
Name: count, dtype: int64
特征 rural 的映射结果:
 rural
0    3918
1    2588
Name: count, dtype: int64
edu 的标签编码结果:
 edu
2    3666
1    1643
0     738
3     451
4       8
Name: count, dtype: int64
SRH编码后的值分布:
 srh
2.0    3046
1.0    1302
3.0     707
4.0     518
0.0     349
Name: count, dtype: int64
特征 hibpe 的二值编码结果:
 hibpe
0    3860
1    2646
Name: count, dtype: int64
特征 diabe 的二值编码结果:
 diabe
0    5793
1     713
Name: count, dtype: int64
特征 cancre 的二值编码结果:
 cancre
0    6399
1     107
Name: count, dtype: int64
特征 lunge 的二值编码结果:
 lunge
0    5329
1    1177
Name: count, dtype: int64
特征 hearte 的二值编码结果:
 hearte
0    5086
1    1420
Name: count, dtype: int64
特征 stroke 的二值编码结果:
 stroke
0    6131
1     375
Name: count, dtype: int64
特征 psyche 的二值编码结果:
 psyche
0    6346
1     160
Name: count, dtype: int64
特征 arthre 的二值编码结果:
 arthre
0    3579
1    2927
Name: count, dtype: int64
特征 dyslipe 的

In [297]:
data_clean.head()

,ID,gender,marry,rural,srh,adlab_c,hibpe,diabe,cancre,lunge,...,age,edu,memeory,executive,total_cognition,chronic,ins,pension,iadl,ea001s9
0,10104105001,0,1.0,0,1.0,1.0,0,0,0,0,...,70.0,2,2.500000,3.000000,5.500000,1,1,1,0.0,0
1,10104105002,1,1.0,0,0.0,0.0,1,0,0,0,...,72.0,0,3.500000,9.000000,12.500000,1,0,0,0.0,0
2,10104107001,1,1.0,0,NaN,0.0,1,0,0,0,...,83.0,2,3.400021,8.393857,12.040004,1,1,1,0.0,0
3,10104107002,0,1.0,0,NaN,0.0,0,0,0,0,...,85.0,2,3.400021,8.393857,12.040004,0,1,1,1.0,0
4,10104110002,1,1.0,0,3.0,0.0,1,0,0,0,...,69.0,2,0.500000,6.000000,6.500000,1,1,1,2.0,0


相关性分析

In [299]:
print("\n进行相关性分析...")
# 选择数值型变量（排除ID和非数值列）
numeric_data = data_clean.select_dtypes(include=['int64', 'float64']).drop(columns=['ID'])

# 确保adlab_c存在于数据中
if 'adlab_c' in numeric_data.columns:
    # 提取目标变量
    target = numeric_data['adlab_c']
    # 计算其他所有特征与adlab_c的相关系数（自动排除目标变量自身）
    adlab_corr = numeric_data.drop(columns=['adlab_c']).corrwith(target).sort_values(ascending=False)
    
    # 打印结果
    print("与adlab_c的相关系数（前10个）:")
    print(adlab_corr.head(10))
    
    corr_threshold = 0.05
    high_corr_features = adlab_corr[abs(adlab_corr) > corr_threshold].index.tolist()
    print("\n相关性较高的特征（前10个）:", high_corr_features[:10])
else:
    print("错误：数据中未找到目标变量 adlab_c！")
#显著性检验：检查相关系数是否统计显著（通常 p<0.05）

p_values = [pearsonr(numeric_data[col], target)[1] for col in high_corr_features]


进行相关性分析...
与adlab_c的相关系数（前10个）:
iadl             0.699175
cesd10           0.306934
disability       0.275004
memrye           0.213346
stroke           0.191231
hospital_time    0.170307
age              0.169428
hospital         0.163991
arthre           0.162029
chronic          0.114037
dtype: float64

相关性较高的特征（前10个）: ['iadl', 'cesd10', 'disability', 'memrye', 'stroke', 'hospital_time', 'age', 'hospital', 'arthre', 'chronic']


ValueError: array must not contain infs or NaNs